In [6]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd
import csv

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = ["ps_packetloss-2018.2*","ps_trace-2018.2*"]

my_query = {}

# CHANGE SO IT RETREIVES ONLY UNIQUE hashes.

# sS='UC'
# srcSiteOWDServer = "192.170.227.160"
# srcSiteThroughputServer = "192.170.227.162"

sS='CERN-PROD'
srcSiteOWDServer = "128.142.223.247"
srcSiteThroughputServer = "128.142.223.246"

# dS='IU'
# destSiteOWDServer = "149.165.225.223"
# destSiteThroughputServer = "149.165.225.224"

# dS='UIUC'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.96.9"

# dS='ICCN'
# destSiteOWDServer = "72.36.96.4"
# destSiteThroughputServer = "72.36.126.132"

dS='pic'
destSiteOWDServer = "193.109.172.188"
destSiteThroughputServer = "193.109.172.187"
'''
#grab user system current date
current_date = datetime.today()

#correct 1 digit days/months to DD/MM format
user_yr = str(current_date.year)
user_month = str(current_date.month)
if len(user_month) < 2:
    user_month = '0' + user_month
user_day = str(current_date.day)
if len(user_day) < 2:
    user_day = '0' + user_day
end_date = user_yr + user_month + user_day + 'T000000Z'

#subtract 1 week from current date
begin_raw = current_date - timedelta(weeks=10)

#do same corrections as above
start_yr = str(begin_raw.year)
start_month = str(begin_raw.month)
if len(start_month) < 2:
    start_month = '0' + start_month
start_day = str(begin_raw.day)
if len(start_day) < 2:
    start_day = '0' + start_day
start_date = start_yr + start_month + start_day + 'T000000Z'
'''

start_date = '20180215T000000Z'
end_date = '20180215T002059Z'

my_query = {
    "size":1,
    "_source": {
        "include": ['timestamp' ,'src','dest','hops','hash','packet_loss' ]
    },
    'query':{
        'bool':{
            'must':[
                {'range': {'timestamp': {'gte': start_date, 'lt': end_date}}},
#                {'term': {'_type': 'traceroute'}},
#                         {'bool':
#                             {'should':[
#                                 {'term': {'src': srcSiteOWDServer}},
#                                 {'term': {'src': srcSiteThroughputServer}},
#                                 {'term': {'src': destSiteOWDServer}},
#                                 {'term': {'src': destSiteThroughputServer}}
#                             ]}
#                         }
#                         ,
#                         {'bool':
#                             {'should':[
#                                 {'term': {'dest': destSiteOWDServer}},
#                                 {'term': {'dest': destSiteThroughputServer}},
#                                 {'term': {'dest': srcSiteOWDServer}},
#                                 {'term': {'dest': srcSiteThroughputServer}}
#                             ]}
#                         }
            ]

        }
    },
    "aggs": {
        "grouped_by_hash": {
          "terms": {  "field": "hash", "size":1000 }, #
          "aggs": {
              "top_hash_hits": {
                  "top_hits": {
                      "sort": [ { "_score": { "order": "desc" } } ],
                      "size": 1
                  }
              }
          }
       }
    }
}
results = es.search(body=my_query, index=my_index, request_timeout=10000)
#results2 = es.search(body=my_query, index=my_index[1], request_timeout=10000)
#print(results['aggregations']['grouped_by_hash']['buckets'])
#print("whuhwwhhwfhiw")
print(results)
#print("IWFUQG8FOWHIFHOIWFQOF")
#print(results2)
print('done')

{'_shards': {'successful': 280, 'failed': 0, 'total': 280, 'skipped': 270}, 'timed_out': False, 'aggregations': {'grouped_by_hash': {'doc_count_error_upper_bound': 5, 'sum_other_doc_count': 15047, 'buckets': [{'doc_count': 60, 'top_hash_hits': {'hits': {'total': 60, 'max_score': 1.0, 'hits': [{'_type': 'doc', '_id': 'AWGWzEnJKTZJJXLfAPp8', '_source': {'dest': '193.60.193.3', 'src_site': 'UKI-LT2-UCL-HEP', 'src_VO': 'ATLAS', 'dest_site': 'UKI-SCOTGRID-DURHAM', 'src_host': 'lcg-sonar01.hep.ucl.ac.uk', 'dest_VO': 'ATLAS', 'ipv6': False, 'MA': '193.60.193.3', 'hops': ['128.40.4.245', '128.40.20.149'], 'timestamp': 1518653251000, 'dest_host': 'perfmon.dur.scotgrid.ac.uk', 'dest_production': True, 'max_rtt': 0.2, 'rtts': [0.2, 0.2], 'n_hops': 2, 'hash': 3161852279277434577, 'src_production': True, 'ttls': [1, 2], 'src': '128.40.4.25'}, '_score': 1.0, '_index': 'ps_trace-2018.2.15'}]}}, 'key': 3161852279277434577}, {'doc_count': 52, 'top_hash_hits': {'hits': {'total': 52, 'max_score': 1.0, 'h